# Filtro de Spam 

Este projeto, proposto na disciplina de **Aprendizado de Máquina**, tem como objetivo classificar e-mails como spam ou legítimos através do algoritmo _Naive Bayes_.

Estaremos utilizando o [dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) da _UCI Machine Learning Repository_.

O trabalho está dividido da seguinte forma:
1. Implementar os tópicos em "ToDo";
2. Explicar o que significa "Bag of Words";
3. Explicar a diferença entre especificidade e sensitividade;
4. Implementar o Naive Bayes e comparar com um dos algoritmos vistos no ML tour;
5. Fazer uma lista dos prós e contras do Naive Bayes e do algoritmo escolhido.

## Implementar ToDos

> **Instrução:** Responder/implementar corretamente todos os tópicos em "ToDo"

Este tópico foi inteiramente resolvido dentro de [Bayesian_Inference.ipynb](Bayesian_Inference.ipynb)

## Bag of Words

> **Instrução:** Explicar o que significa o "Bag of Words"

_Bag of Words_ (BoW) é uma técnica utilizada para representar dados de texto como um vetor numérico. Ela essencialmente conta a frequência de palavras ignorando a ordem em que elas ocorrem.

De forma exemplificada, considere as seguintes frases:
- **Frase 0:** O tempo perguntou ao tempo quanto tempo o tempo tem
- **Frase 1:** Uma breve história do tempo

Podemos visualizá-las em forma de matriz: cada linha `i` representa uma frase, enquanto a coluna `j` representa uma palavra. A entrada `ij` da matriz guarda a frequência em que a palavra `j` aparece na frase `i`. 

Ignorando a diferença entre maiúsculas e minúsculas, teríamos:

|   | o | tempo | perguntou | ao | quanto | tem | uma | breve | história | do |
|:-:|:-:|:-----:|:---------:|:--:|:------:|:---:|:---:|:-----:|:--------:|:--:|
| **0** | 2 | 4 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 |
| **1** | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 1 |

## Especificidade x Sensitividade

> **Instrução:** Explicar a diferença entre especificidade e sensitividade. Dê dois exemplos práticos, uma em que a especificidade parece ser mais adequada do que a sensitividade e vice-versa

A **sensitividade** de um teste, também chamada de taxa de verdadeiro positivo (TPR) ou  _recall_, é a taxa de positivos que foram corretamente identificados como positivos

A sensibilidade pode ser calculada da seguinte forma:

```
sensibilidade = #[verdadeiros positivos] / (#[verdadeiros positivos] + #[falsos negativos])
```

Já a **especificidade** de um teste, também chamada de taxa de verdadeiro negativo (TNR), é a taxa de negativos que foram corretamente identificados como negativos, i.e, 

```
especificidade = #[verdadeiros negativos] / (#[verdadeiros negativos] + #[falsos positivos])
```

O ideal é que tanto a sensitividade quanto a especificidade tenham os maiores valores possível, mas nem sempre isso é possível. A escolha entre algoritmos que priorizam sensibilidade ou especificidade depende diretamente da natureza do problema.

Por exemplo, em um teste para detecção de câncer, é importante priorizar a sensitividade para garantir que a maior parte dos casos positivos seja identificada. Perder um positivo (paciente com câncer) por falta de tratamento seria muito grave.

Em contrapartida, um filtro de spam deve priorizar a especificidade. Receber um spam é menos pior do que perder uma mensagem importante.

## Naive Bayes x Regressão logística

> **Instrução:** Implementar o Naive Bayes conforme indica o roteiro e comparar com um (1) dos algoritmos vistos no ML tour, justificando a escolha do melhor modelo. É para comparar o Naive Bayes com um e apenas um algoritmo

Como visto em [Bayesian_Inference.ipynb](Bayesian_Inference.ipynb), os resultados que tivemos com o _Naive Bayes_ foram os seguintes:

In [1]:
nb_accuracy = 0.9885139985642498

nb_precision = 0.9720670391061452

nb_recall = 0.9405405405405406

nb_f1 = 0.9560439560439562

nb_specificity = 0.9958609271523179

Faremos o mesmo com a Regressão Logística para fins de comparação.

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_table('collection/SMSSpamCollection', 
                    sep='\t',
                    names=['label', 'message'])

df['label'] = df['label'].map({'ham': 0, 'spam': 1})

X_train, X_test, y_train, y_test = train_test_split(df['message'],
                                                    df['label'],
                                                    random_state=1)

count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)


In [32]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(training_data, y_train)
predictions = logistic_regression.predict(testing_data)

Calcularemos as métricas da Regressão Logística:

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print('Accuracy score: ', format(accuracy_score(y_test, predictions)))

print('Precision score: ', format(precision_score(y_test, predictions)))

print('Recall score: ', format(recall_score(y_test, predictions)))

print('F1 score: ', format(f1_score(y_test, predictions)))

lr_tn, lr_fp, lr_fn, lr_tp = confusion_matrix(y_test, predictions).ravel()
lr_specificity = lr_tn / (lr_tn+lr_fp)
print('Specificity: ', format(lr_specificity))

Accuracy score:  0.9885139985642498
Precision score:  0.9567567567567568
Recall score:  0.9567567567567568
F1 score:  0.9567567567567568
Specificity:  0.9933774834437086


### Comparação dos resultados

**1. Accuracy**

- **Naive Bayes:** 0.9885139985642498
- **Regressão Logística:** 0.9877961234745154

Esta métrica mede a proporção de previsões corretas feitas pelo modelo em todo o conjunto de dados. Note que ambos tiveram resultados próximos, porém o Naive Bayes teve uma pequena vantagem. 

**2. Precision**

- **Naive Bayes:** 0.9720670391061452
- **Regressão Logística:** 0.9941176470588236

_Precision_ mede a proporção de previsões de verdadeiros positivos entre todas as previsões positivas. Uma precisão maior significa menos falsos positivos, ou seja, menos e-mails legítimos serão erroneamente classificados como spam.

Nesse caso, a Regressão Logística demonstrou melhor desempenho.

**3. Recall**

- **Naive Bayes:** 0.9405405405405406
- **Regressão Logística:** 0.9135135135135135

O _recall_ (sensitividade) mede a proporção de positivos que foram identificadas corretamente, isto é, um _recall_ mais alto significa que o modelo é melhor em identificar todos os e-mails de spam.

Naive Bayes tem um recall maior em comparação com a Regressão Logística. 

**4. F1**

- **Naive Bayes:** 0.9560439560439562
- **Regressão Logística:** 0.9521126760563381

_F1_ equilibra em uma única métrica o _recall_ e a precisão. Ambos algorítmos atingem valores semelhantes, porém Naive Bayes está levemente acima.

**5. Specificity**

- **Naive Bayes:** 0.9958609271523179
- **Regressão Logística:** 0.99917218543046361

Como já dito anteriormente, a especificidade mede a taxa de negativos corretamente denominados como tal. Note que a Regressão Logística levou vantagem, isso significa que ela foi melhor em classificar e-mails legítimos como legítimos.

### Qual escolher?

Ambos os modelos possuem bons desempenhos em todas as métricas (> 90%). Levando em consideração que especificidade e _precision_ são mais importantes nesse contexto, visto que queremos evitar que e-mails legítimos sejam erroneamente classificicados como spam, a Regressão Logística é a melhor escolha. Embora o Naive Bayes leve vantagem em algumas métricas, a diferença não é tão expressiva.

## Prós e contras

> **Instrução:** Você deverá fazer uma lista dos prós e contras do Naive Bayes e do algoritmo escolhido. Você deve explicar o resultado obtido com base nas características dos dois algoritmos avaliado. Você teria algum insight relevante para me apresentar?

### Naive Bayes

**Pros**

- **Baixo _overfitting_:** esse modelo causa pouco _overfitting_ em comparação com outros algoritmos, por conta disso, as 4 métricas possuem bons valores no geral;

- **Bom com pequenos conjuntos de dados:** Naive Bayes é um algoritmo simples e eficiente, se dando muito bem com conjuntos menores de dados. Este é o nosso caso, já que estamos trabalhando com uma lista de apenas 5574 mensagens.


**Cons**

- **Assume independência:** Naive Bayes assume que todas as palavras são independentes umas das outras, o que normalmente não é verdade. Isso pode levar a um número maior de falsos positivos, reduzindo a precisão;

- **Zero-frequency problem:** isso acontece quando uma variável, no nosso caso uma palavra, está ausente nos dados de treinamento. Por exemplo, se "maçã" não estiver nos dados de treinamento para "spam", sua probabilidade será zero, tornando também a probabilidade geral zero.

### Regressão Logística

**Pros**

- **Bom para classificação binária:** é exatamente esse o propósito do filtro de spam: classificar as mensagens entre spam ou ham;

- **Simples de interpretar:** isso acontece pois a probabilidade é calculada como uma combinação linear das variáveis.

**Cons**

- **Maior _overfitting_:** a Regressão Logística é mais propensa a _overfitting_ em relação ao Naive Bayes. Essa pode ser uma das causas da diferença entre Accuracy e Recall ser maior quando comparado ao Naive Bayes;

- **Assume Separabilidade Linear:** se as mensagens de spam e ham não forem claramente separáveis com uma linha reta, a regressão logística pode não capturar tais nuances.

### Insights

**Naive Bayes:** Esse modelo obteve melhor _recall_, o que significa que está detectando melhor as mensagens de spam. Pode ser uma boa escolha se detectá-los for mais importante, mesmo que alguns falsos positivos passem despercebídos.

**Regressão logística:** Já esse modelo possui maior precisão e especificidade, então ele é melhor em evitar falsos positivos (rotulando não spam como spam). É mais adequado se minimizar os falsos positivos for uma prioridade, mesmo que isso signifique que algumas mensagens de spam passem como legítimas.

Particularmente, acredito que minimizar que e-mails legítimos sejam classificados como spam é mais importante do que o contrário; focar em maior especificidade se torna melhor.